** This code requires a Bria API token: https://platform.bria.ai/register 

In [1]:
import os
headers = {
  "Content-Type": "application/json",
  "api_token": os.getenv("BRIA_API_TOKEN")  # Ensure BRIA_API_TOKEN is set in your environment variables
}

In [2]:
from demo_utils import *

# Welcome to Bria: AI-Powered Visual Content Generation

Bria is a visul-Gen-AI platform for builders, designed for commercial use and powered exclusively by licensed data.

This demo will showcase how developers can leverage Bria's APIs to build tools that enable their users to generate and modify brand-consistent visuals at scale.

In [ ]:
display_images([Image.open("./visuals/bria_intro1.jpg"), Image.open("./visuals/bria_intro2.jpg")], resize=800)

## Leveraging Gen-AI for Brand Content Creation

We'll use **Bria's API** to show how a developer can easily **build** tools for creating and editing **controlled**, **on-brand** visuals **at scale**.

A strong brand identity includes visual features such as:

- **Color Palette**
- **Style & Mood**
- **Recurring Characters ("mascot") & Themes** 
- **Fonts** 
- **Logo**

The example below showcases how such features are present in the Bria brand:

In [ ]:
display_images([Image.open("./visuals/bria_brand_example.png")], resize=700)

## **Text-to-Image**: AI-Generated Visuals

Visual GenAI starts with text-to-image generation. 

Below, we define a function to generate images from a prompt using Bria's **text-to-image API**:

https://docs.bria.ai/image-generation/endpoints/text-to-image-fast

In [3]:
def text_to_image(prompt, num_results=3, model_version="2.3", fast=False, seed=42):
    """
    Generate AI-powered images from text prompts using Bria's API.
    
    Parameters:
    - prompt (str): The text description of the desired image.
    - model_version (str): foundation model version to use.
    - num_results (int): Number of images to generate.
    - fast (bool): Whether to use the fast generation endpoint.
    - seed (int): Seed for reproducibility.
    
    Returns:
    - List of generated image objects.
    """
    
    base_url = f"https://engine.prod.bria-api.com/v1/text-to-image/base/{model_version}"
    if fast:
        base_url = f"https://engine.prod.bria-api.com/v1/text-to-image/fast/{model_version}"

    payload = {
        "prompt": prompt, 
        "seed": seed, 
        "num_results": num_results,
        "sync": True, 
    }

    response = requests.post(base_url, json=payload, headers=headers)
    responses = response.json().get("result", [{}])
    print(responses[0].get("warning",""))
    image_urls = [x.get("urls", [None])[0] for x in responses]

    return return_images_from_urls(image_urls)

In [ ]:
## Example: Generating an Image with Bria
# The following example demonstrates how Bria's API generates AI-driven visuals from text descriptions.

prompt = 'A 3D render of a purple skinned elephant, over white background'

images = text_to_image(prompt, num_results=3)
display_images(images, f"prompt: {prompt}")

**Bria’s Responsible AI**

Bria ensures content safety by preventing the generation of copyrighted material. For example: 

In [ ]:
prompt = "A 3D render of a purple skinned elephant, that resembels Disney's Dumbo"
images = text_to_image(prompt)
display_images(images, f"prompt: {prompt}")

When generating the same prompt with Flux we get the following result:

In [ ]:
display_images([Image.open("./visuals//flux_dumbo.png")])

### Reproducing The Brand Mascot

We want to enable users to accurately create visuals using their brand assets, such as the following examples of the **Bria Elephant**:

In [ ]:
# load sample of bria elephant oiriginal images

bria_bear_dir = "briaphant"
images = [Image.open(f"{bria_bear_dir}/{f}") for f in os.listdir(bria_bear_dir) if "png" in f][:4]
display_images(images, "Bria Elephant - Originals", resize = 1000, font_size=40)

We need to add more components to the text-to-image generation to increase the controlability. We'll start by introducing **Control-Nets**.

## Image Guidance for Controlled Generation
We can add **structural control** using an input image to generate variations of that image, using the following Control Nets which were trained on-top of Bria's foundation text-to-image model:
- Canny
- Depth

In [ ]:
display_images([Image.open("./visuals/control_nets.png")], resize=800)

We'll choose one of the Bria Elephant original images and add structural image guidance to the generation using our trained ControlNets.

We can use Bria's **Reimagine API** which combines both ControlNets to reproduce the structure of the input image, while allowing changes through the textual prompt:

https://docs.bria.ai/image-generation/endpoints/reimagine-structure-reference

In [4]:
def text_to_image_with_guidance(prompt, guidance_image, num_results=3, fast=False, seed=42):
    
    base_url = "https://engine.prod.bria-api.com/v1/reimagine"

    payload = {
        "prompt": prompt, 
        "seed": seed, 
        "num_results": num_results,
        "sync": True, 
        "structure_image_file": pil_image_to_base64(guidance_image),
        "fast": fast
    }
    
    response = requests.post(base_url, json=payload, headers=headers)
    responses = response.json().get("result", [{}])
    image_urls = [x.get("urls", [None])[0] for x in responses]

    return return_images_from_urls(image_urls)

In [ ]:
#Input:
prompt = 'A 3D render of a purple skinned elephant, over white background'
guidance_image = Image.open("./briaphant/bria_1afcb261_2000_49eb_a908_3656fd9a67fd_4.png")

images = text_to_image_with_guidance(prompt, guidance_image, num_results=2)
display_images([guidance_image], 'Input Image')
display_images(images, f"prompt: {prompt}")

We can use Reimagine to change the color of the Bria Elephant:

In [ ]:
images = []
for color in ['blue', 'green', 'brown', 'rainbow colored']:
    prompt = f'A 3D render of a {color} skinned elephant, over white background'
    
    guidance_image = Image.open("./briaphant/bria_1afcb261_2000_49eb_a908_3656fd9a67fd_4.png")

    image = text_to_image_with_guidance(prompt, guidance_image, num_results=1)
    images.append(image[0])
display_images(images, 'prompt: A 3D render of a {color} skinned elephant, over white background')

In some cases this structural control is not enough. We want to enable our users to teach the model to generate a more accurate and varied represenation of this character.

We will allow this by enabling our users to **fine-tune** our foundation model using the original brand images they own.

## Tailored-Generation - Fine-Tuning with LoRA
### Training

We can fine-tune Bria's foundation model using the existing images of the Bria Elephant. For fine-tuning we would use Bria's **4B-Adapt** model, which is designed to provide exceptional fine-tuning capabilities for commercial use: https://huggingface.co/briaai/BRIA-4B-Adapt


We fine-tune using LoRA for easier deployment of each such fine-tuned model. 


Please refer to the following examples:

- Training using Bria's automatic Tailored-Gen API: gtc_demo_fine_tune_api.ipynb

- Training on-prem using Bria's foundation model weights on HF: gtc_demo_fine_tune_on_prem.ipynb

### Inference

We'll use Bria's Generate Image - Tailored Model API:

https://docs.bria.ai/tailored-generation/endpoints/text-to-image-tailored

In [5]:
def tailored_gen(prompt, tailored_model, num_results=3, seed=42):
    
    base_url = f"https://engine.prod.bria-api.com/v1/text-to-image/tailored/{tailored_model}"

    payload = {
        "prompt": prompt, 
        "seed": seed, 
        "num_results": num_results,
        "sync": True, 
    }

    response = requests.post(base_url, json=payload, headers=headers)
    responses = response.json().get("result", [{}])
    image_urls = [x.get("urls", [None])[0] for x in responses]

    return return_images_from_urls(image_urls)

In [ ]:
prompt = "holding a pink cocktail in its trunk and wearing a small pink party hat"
tailored_model = 10920
elephant_image = tailored_gen(prompt, tailored_model, num_results=1)[0]
elephant_image.save('./api_results/elephant_image.jpg')
display_images([elephant_image], f"prompt: \n{prompt}", font_size=10)

Great. Now let's place this festive character in a proper location by generating on-brand backgrounds.

We want the background to adhere to the brand style as well, so we'll use a tailored model trained on the following brand style images:

In [ ]:
# load sample of bria style oiriginal images
bria_style_dir = "bria_style"

images = [Image.open(f"{bria_style_dir}/{f}") for f in os.listdir(bria_style_dir)][:4]
display_images(images, "Bria Style - Originals", resize = 1000, font_size=40)

We used an LLM to write a few prompts for background images that could be relevant for this character in a festive event. 

Let's generate 1 example from each, using the brand style tailored model.

In [9]:
background_prompts = [
 'A tropical beach at sunset with palm trees, soft waves, and string lights, perfect for a relaxing party vibe',
 'A winter wonderland party with twinkling snowflakes, icy-blue lighting, and festive holiday purple decorations creating a cozy atmosphere',
 'A cosmic galaxy party with glowing planets, swirling nebulas, and a dance floor that looks like the surface of the moon.',
 'A lively party venue with colorful decorations, balloons, streamers, and warm lighting, creating a fun and festive atmosphere',
 'A carnival-themed party with bright lights, a Ferris wheel in the background, colorful booths, and festive decorations',
 'A futuristic space party with floating balloons, glowing neon decorations, and a starry galaxy sky in the background',
 "A retro '80s party with neon colors, arcade machines, a checkered dance floor, and a boombox playing classic hits.",
 'A jungle adventure party with tropical foliage, tiki torches, tribal drums, and exotic animals hidden in the background.',
 'A rooftop sunset cocktail party with stylish lounge seating, golden hour lighting, and a panoramic city skyline view.'
 ]

In [ ]:
bg_originals_dir = "./api_results/bg_gen/bg_originals"
os.makedirs(bg_originals_dir, exist_ok=True)

all_bg_images = []
for i, prompt in enumerate(background_prompts):
    tailored_model = 10900
    bg_image = tailored_gen(prompt, tailored_model, num_results=1)[0]
    bg_image.save(f'{bg_originals_dir}/{i}.png')
    all_bg_images.append(bg_image)

all_bg_images = [Image.open(f'{bg_originals_dir}/{i}.png') for i in range(len(background_prompts))]
display_images(all_bg_images[:3], f"Generated Backgrounds from Bria Style Tailored Model", resize = 256)
display_images(all_bg_images[3:6], resize = 256)
display_images(all_bg_images[6:9], resize = 256)

## Background Genreation (by Reference Image)
Next, we want to use those brand style backgrounds as inspiration for new background for our festive Bria Elephant. 

We'll use Bria's Generate Background API:


https://docs.bria.ai/image-editing/endpoints/background-replace

In [13]:
def generate_bg_by_image(foreground_image, bg_image, num_results=1, seed=42, padding=[0, 0, 0, 0]):
    
    base_url = "https://engine.prod.bria-api.com/v1/background/replace"

    payload = {
        "file": pil_image_to_base64(foreground_image),
        "ref_image_file": pil_image_to_base64(bg_image),
        "sync": True,
        "placement_type": "manual_padding",
        "num_results": num_results,
        "seed": seed,         
        "padding_values": padding
    }

    response = requests.post(base_url, json=payload, headers=headers)
    responses = response.json().get("result", [{}])
    image_urls = [x[0] for x in responses]

    return return_images_from_urls(image_urls)

In [ ]:
bg_generations_dir = "./api_results/bg_gen/bg_generations"
os.makedirs(bg_generations_dir, exist_ok=True)

all_bg_images = [Image.open(f'{bg_originals_dir}/{i}.png') for i in range(len(background_prompts))]

foreground_image = Image.open('elephant_image.jpg')
resize_scale = 0.25 # we'll make the elephant 25% smaller to fit into the background images

# we want the elephant to be positioned in the bottom right corner of the background image, so we'll add padding accordingly
width_padding = int(foreground_image.size[0]*resize_scale)
height_padding = int(foreground_image.size[1]*resize_scale)
bottom_right_location = [width_padding, 0, height_padding, 0]

all_images = []
for i, bg_img in enumerate(all_bg_images):

    new_bg_img = generate_bg_by_image(foreground_image, bg_img, num_results=1, padding=bottom_right_location)
    new_bg_img[0].save(f'{bg_generations_dir}/{i}.png')
    all_images.append(new_bg_img[0])

# all_images = [Image.open(f'/home/ubuntu/spring/demo_gtc/bg_gen/bg_generations/{i}.png') for i in range(len(background_prompts))]

display_images(all_images[:3], f"Replaced Backgrounds with Reference Images", resize = 256)
display_images(all_images[3:6], resize = 256)
display_images(all_images[6:9], resize = 256)

## Image Editing: Generative Fill

Focusing on the first 3 outputs, let's fix some content issues by replacing or adding content using Bria's Generative-Fill API:

https://docs.bria.ai/image-editing/endpoints/gen-fill

In [53]:
def gen_fill(original_image, mask, prompt, num_results=1, seed=42):
    
    base_url = "https://engine.prod.bria-api.com/v1/gen_fill"

    payload = {
        "file": pil_image_to_base64(original_image),
        "mask_file": pil_image_to_base64(mask.convert('RGB')),
        "mask_type": "manual",
        "prompt": prompt,
        "num_results": num_results,
        "sync": True,
        "seed": seed,
    }

    response = requests.post(base_url, json=payload, headers=headers)
    responses = response.json().get("urls", [])
    image_urls = [x for x in responses]

    return return_images_from_urls(image_urls)


The first image is a beach scene, let's replace the party hat with some beachwear. We'll use a mask around the hat we want to replace, and specifiy the new content in "prompt":

In [ ]:
input_image = Image.open('./api_results/bg_gen/bg_generations/0.png')
mask = Image.open('./api_results/masks/hat_mask.png')
prompt = "a straw hat"

display_mask(mask, input_image)

output_var1 = gen_fill(input_image, mask, prompt)[0]
output_var1.save(f'./api_results/bg_gen/bg_generations/0_fixed.png')
display_images([output_var1], f"prompt: {prompt}")

Now let's replace the hat in the second image to something more suitable for winter festivities:

In [ ]:
input_image = Image.open(f'./api_results/bg_gen/bg_generations/1.png')
mask = Image.open('./api_results/masks/hat_mask.png')
prompt = "a santa hat"

output_var2 = gen_fill(input_image, mask, prompt)[0]
output_var2.save(f'./api_results/bg_gen/bg_generations/1_fixed.png')
display_images([output_var2], f"prompt: {prompt}")

We can also add a christmas tree in the brand colors

In [ ]:
input_image = Image.open(f'./api_results/bg_gen/bg_generations/1_fixed.png')
mask = Image.open('./api_results/masks/tree_mask.png')
prompt = "a lush christmas tree with purple and pink ornaments"

display_mask(mask, input_image)

output_var2 = gen_fill(input_image, mask, prompt)[0]
output_var2.save(f'./api_results/bg_gen/bg_generations/1_fixed.png')
display_images([output_var2], f"prompt: \n{prompt}")

## Image Editing: Eraser
And finally, the summer cocktail doesn't align with the winter theme, so let's remove it. 

We'll use Bria's Eraser API:

https://docs.bria.ai/image-editing/endpoints/eraser

In [56]:
def erase(original_image, mask):
    
    base_url = "https://engine.prod.bria-api.com/v1/eraser"

    payload = {
        "file": pil_image_to_base64(original_image),
        "mask_file": pil_image_to_base64(mask.convert('RGB')),
        "mask_type": "manual",
        "sync": True,
    }

    response = requests.post(base_url, json=payload, headers=headers)
    responses = response.json().get("result_url", '')
    image_urls = [responses]

    return return_images_from_urls(image_urls)


In [ ]:
input_image = Image.open(f'./api_results/bg_gen/bg_generations/1_fixed.png')
mask = Image.open('./api_results/masks/cocktail_mask.png')

display_mask(mask, input_image)

output_var2 = erase(input_image, mask)[0]
output_var2.save(f'./api_results/bg_gen/bg_generations/1_fixed.png')
display_images([output_var2])

## Image Editing: Expand

We now have our 3 image varaiations ready. But what if we wanted to use them in adds with different aspect ratios? 

We'll use Bria's Expand Image API to expand the image to different aspect ratios:

https://docs.bria.ai/image-editing/endpoints/image-expansion

In [ ]:
output_var3 = Image.open(f'./api_results/bg_gen/bg_generations/2.png').resize((350, 350))
image_variations = [output_var1, output_var2, output_var3]
display_images(image_variations, resize = 400)

In [53]:
def expand_left(input_image, seed=1000):
    org_width, org_height = input_image.size
    
    base_url = "https://engine.prod.bria-api.com/v1/image_expansion"

    payload = {
        "file": pil_image_to_base64(input_image),
        "canvas_size": [
            org_width*2,
            org_height
        ],
        "original_image_size": [
            org_width,
            org_height
        ],
        "original_image_location": [
            org_width,
            0
        ],
        "seed": seed
    }

    response = requests.post(base_url, json=payload, headers=headers)
    responses = response.json().get("result_url", '')
    image_urls = [responses]

    return return_images_from_urls(image_urls)

def expand_top(input_image, seed=1000):
    org_width, org_height = input_image.size
    
    base_url = "https://engine.prod.bria-api.com/v1/image_expansion"

    payload = {
        "file": pil_image_to_base64(input_image),
        "canvas_size": [
            org_width,
            org_height*2
        ],
        "original_image_size": [
            org_width,
            org_height
        ],
        "original_image_location": [
            0,
            org_height
        ],
        "seed": seed
    }

    response = requests.post(base_url, json=payload, headers=headers)
    responses = response.json().get("result_url", '')
    image_urls = [responses]

    return return_images_from_urls(image_urls)


In [ ]:
expanded_images_left = []
for image_var in image_variations:
    expanded_images_left.append(expand_left(image_var)[0])

display_images(expanded_images_left, resize = 500)

In [ ]:
expanded_images_top = []
for image_var in image_variations:
    expanded_images_top.append(expand_top(image_var)[0])

display_images(expanded_images_top, resize = 800)

In [ ]:
display_images(image_variations)
display_images([pad_image_to_square(x) for x in expanded_images_top])
display_images([pad_image_to_square(x) for x in expanded_images_left])

In [13]:
#### increase res!